In [1]:
# Importing the libraries
from pathlib import Path

import pandas as pd
import numpy as np
#from tensorflow import keras
#from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
base_csv = Path('data/clean_frame.csv')
test_csv = Path('data/test_frame.csv')

In [7]:
base_df = pd.read_csv(base_csv, index_col='ind')
base_df.columns

Index([&#39;ID&#39;, &#39;Код_группы&#39;, &#39;Год_Поступления&#39;, &#39;Год_Окончания_УЗ&#39;, &#39;Пособие&#39;,
       &#39;Общежитие&#39;, &#39;Наличие_Матери&#39;, &#39;Наличие_Отца&#39;, &#39;Опекунство&#39;, &#39;Село&#39;,
       &#39;Иностранец&#39;, &#39;КодФакультета&#39;, &#39;СрБаллАттестата&#39;, &#39;Статус&#39;, &#39;male&#39;,
       &#39;female&#39;, &#39;sex_summ&#39;, &#39;birth_year_int&#39;, &#39;basis&#39;, &#39;language&#39;, &#39;country&#39;,
       &#39;region&#39;, &#39;city&#39;, &#39;parents_country&#39;],
      dtype=&#39;object&#39;)

In [3]:
test_df = pd.read_csv(test_csv, index_col='ind')
test_df.columns

Index([&#39;ID&#39;, &#39;Код_группы&#39;, &#39;Год_Поступления&#39;, &#39;Год_Окончания_УЗ&#39;, &#39;Пособие&#39;,
       &#39;Общежитие&#39;, &#39;Наличие_Матери&#39;, &#39;Наличие_Отца&#39;, &#39;Опекунство&#39;, &#39;Село&#39;,
       &#39;Иностранец&#39;, &#39;КодФакультета&#39;, &#39;СрБаллАттестата&#39;, &#39;male&#39;, &#39;female&#39;,
       &#39;sex_summ&#39;, &#39;birth_year_int&#39;, &#39;Статус&#39;, &#39;basis&#39;, &#39;language&#39;, &#39;country&#39;,
       &#39;parents_country&#39;],
      dtype=&#39;object&#39;)

In [ ]:
base_df = base_df.fillna(0)


In [4]:
#test_df.isna().any()
test_df = test_df.fillna(0)

In [5]:
columns = ['Наличие_Матери', 'Село', 'Иностранец', 'СрБаллАттестата', 'Статус', 'male', 'basis','language','country', 'КодФакультета', 'Общежитие', 'parents_country']
#columns = base_df.columns.to_list()
priznaki = len(columns)
priznaki

12

In [8]:
frame_to_train = base_df.loc[:, columns]
frame_to_test = test_df.loc[:, columns]

In [9]:
result_key = {
    -1: 0,
    3: 1,
    4: 2,
}

In [10]:
for key, value in result_key.items():
    frame_to_train.loc[frame_to_train['Статус'] == key, 'Статус'] = value

frame_to_train.groupby('Статус').count()

,Наличие_Матери,Село,Иностранец,СрБаллАттестата,male,basis,language,country,КодФакультета,Общежитие,parents_country
Статус,,,,,,,,,,,
0,614,614,614,614,614,614,614,614,614,614,614
1,4721,4721,4721,4721,4721,4721,4721,4721,4721,4721,4721
2,8249,8249,8249,8249,8249,8249,8249,8249,8249,8249,8249


In [11]:
train, test = train_test_split(frame_to_train, test_size=0.20)

In [ ]:
train_input = train.drop('Статус', axis=1)
train_output = train['Статус']

In [ ]:
test_input = test.drop('Статус', axis=1)
test_output = test['Статус']

In [ ]:
train_output.info

In [ ]:
input_df_size = train_input.shape[0]

In [ ]:
# Constructing the input
x = train_input.to_numpy()
y = train_output.values
arr = np.transpose(x)

In [ ]:
output = np.ones((1,input_df_size))
output[0] = y
arr_y = output.reshape(input_df_size,1)
arr_y

In [ ]:
# Definig the model
model_better = keras.Sequential([
    keras.layers.Dense(150, input_shape=(priznaki-1,), activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(250, activation='relu'),    
    keras.layers.Dense(3, activation='softmax'),
])

In [ ]:
# Compiling the model
model_better.compile(optimizer='Nadam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

In [ ]:
# fitting the model
model_better.fit(x, y, epochs=100, batch_size=70)

In [ ]:
# Evaluating the model
x_t = test_input.to_numpy()
y_t = test_output.values

model_better.evaluate(x_t, y_t, batch_size=8)

In [ ]:
bet = model_better.evaluate(x_t, y_t, batch_size=8)
bet

In [ ]:
tusa = model_better.predict(x_t)
tusa

In [ ]:
def get_result(answer_arr: np.array) -> int:
    answer_code = np.argmax(answer_arr)

    # list out keys and values separately
    key_list = list(result_key.keys())
    val_list = list(result_key.values())
    
    # print key with val 100
    position = val_list.index(answer_code)
    return  key_list[position], answer_code

In [ ]:
answers = []
for answer_arr in tusa:
    answers.append(get_result(answer_arr)[1])

In [ ]:
test_output

In [ ]:
compare_df = pd.DataFrame(columns=['real', 'calculation'])
compare_df['real'] = y_t
compare_df['calculation'] = answers
compare_df

In [ ]:
compare_df['new'] = np.where((compare_df['real'] == compare_df['calculation']), compare_df['real'], 100)
compare_df.groupby('new').count()

In [ ]:
r = 1 + 386 + 1032 + 320
620 / r

# Чтение и предсказие test dataset

In [12]:
train_input = train.drop('Статус', axis=1)
input_df_size = train_input.shape[0]

In [13]:
# Constructing the input
x_test = train_input.to_numpy()

In [ ]:
predict_arr = model_better.predict(x)
predict_arr

In [ ]:
answers = []
for answer_arr in predict_arr:
    answers.append(get_result(answer_arr)[1])
answers

In [ ]:
compare_df = pd.DataFrame(columns=['real', 'calculation'])
compare_df['real'] = y
compare_df['calculation'] = answers
compare_df